# [2021 gnssrefl short course](https://www.unavco.org/event/2021-gnss-interferometric-reflectometry/)

---

## Homework 2

**Due date:** This homework is to be completed **before** the short course given on October 21. You need to make
sure the software has been properly installed and you have successfully completed the "homework 0" assignment.

**Purpose:** Learn how to measure snow depth levels with gnssrefl using GNSS data 

In [ ]:
import os
import sys
import re
import json
import pandas as pd 
import numpy as np
import seaborn as sns; sns.set_theme(style="whitegrid");
import matplotlib.pyplot as plt

# We are including our repository bin to the system path so that we can import the following python modules
bin_path = os.path.abspath(os.path.join('../../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)
    
import run_gnssrefl 
import gnssrefl_helpers

%matplotlib inline

In [ ]:
#Making sure environment variables are set - this is required to run the gnssrefl code
exists = gnssrefl_helpers.check_environment()
if exists == False:
    gnssrefl_helpers.set_environment()
else:
     print('environment variable ORBITS path is', os.environ['ORBITS'],
          '\nenvironment variable REFL_CODE path is', os.environ['REFL_CODE'],
          '\nenvironment variable EXE path is', os.environ['EXE'])
        
refl_code_loc = os.environ['REFL_CODE']
# import the crx2rnx file which is dependant on your working OS - this is required to run the gnssrefl code
gnssrefl_helpers.download_crx2rnx()

**Station:**
We will be using station **gls1**.
gls1 was installed at [Dye2](http://greenlandtoday.com/dye-2-a-relic-from-a-not-so-distant-past/?lang=en) on the Greenland Ice Sheet in 2011. 
The antenna is mounted on a long pole; approximately 3.5-meter of the pole was above the ice at the time of installation. 
A detailed discussion of the monument and 
data from the station can be found in [Larson, MacFerrin, and Nylen (2020)](https://tc.copernicus.org/articles/14/1985/2020/tc-14-1985-2020.pdf). 
The latest position time series for gls1 can be retrieved 
from the [Nevada Geodetic Laboratory](http://geodesy.unr.edu/gps_timeseries/tenv3/IGS14/GLS1.tenv3). 

As gls1 is on an ice sheet and the ice surface is relatively smooth in all directions, it 
is unlikely that a complicated azimuth mask will be required.

gls1 was originally installed with an elevation mask of 7 degrees, which is suboptimal for reflections research.
Even though the mask was later removed, we will use 7 degrees as the minimum elevation angle for all our analysis.
Similarly, even though the site managers later changed to enable L2C tracking, to ensure that 
a consistent dataset is being used, we will only use L1 data.

Run the cell below (starts with %%html) to view the gnss-reflections webapp geoid functionality to get an idea of its surroundings. You can enter the station coordinates by hand if you know them, but since gls1 is part of a public archive known to geodesists, coordinates have been stored in the webapp. Just type in gls1 for the station name. Make a note of the station latitude, longitude, and ellipsoidal height that is returned by the webapp because you will need it later. 

In [ ]:
station = 'gls1'

# fill in values here from the web app in the cell below
lat = 
long = 
height = 

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/geoid" width="800" height="600"></iframe>

**Using gnssrefl**

Our ultimate goal in this use case is to analyze one year of data. We have chosen the year 2012 because there was a large melt event on the ice sheet. In order to set the proper quality control parameters, we will use quickLook for one day. First we need to translate one day of RINEX data using rinex2snr. We will use day of year 100:

We need to pick up a RINEX file and strip out the SNR data.  We use the <code>rinex2snr</code> for this purpose. 
The only required inputs are the station name (gls1), the year (2012) and day of year (100).

In [ ]:
# run this cell to check out the parameters and default values
run_gnssrefl.rinex2snr?

In [ ]:
year = 
doy = 
# if you would like to pick an archive then you can 
# archive = 'unavco'
# archive = 'sopac'

# if you choose to use an archive then you must add the parameter archive example:
# run_gnssrefl.rinex2snr(station, year, doy, archive=archive)

# otherwise, the only required parameters are station, year, and doy
run_gnssrefl.rinex2snr(station, year, doy)

---
Once you have successfully created a SNR file, run <code>quickLook</code>.

There are two ways to view the quicklook plots in the jupyter notebook version:

* The first way is to set the parameter pltscreen to True (`pltscreen=True`). This will print out the plots that the gnssrefl code makes.

* The second way is to make the plot ourselves so we can better view these plots. In that case we set `pltscreen=False` - which is also the default. Both options are provided in the cells below.

The quicklook plots consist of two graphical representations of the data. The first is 
periodograms for the four geographic quadrants (northwest, northeast, and so on). 
You are looking for nice clean (and colorful) peaks. Color means they have 
passed Quality Control (QC). Gray lines are satellite tracks that failed QC. The second plot summarizes the 
RH retrievals and how the QC metrics look compared to 
the defaults. In this case the x-axis is azimuth in degrees.
[For more details on quicklook output](https://github.com/kristinemlarson/gnssrefl/blob/master/docs/quickLook_desc.md).

In [ ]:
# Plotting using pltscreen=True
values, metrics = run_gnssrefl.quicklook(station, year, doy=doy, pltscreen=True)

In [ ]:
# plotting the quicklook graph periodograms
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(12,10), sharex=True)
fig.suptitle(f'QuickLook: {station}', size=16)

quadrants = ['NW', 'NE', 'SW', 'SE']
quadrant_labels = ['Northwest','Northeast', 'Southwest', 'Southeast']

for i, ax in enumerate(axes.flat):
    quad = quadrants[i]
    for fail_satellite in values[f'f{quad}']:
        g = sns.lineplot(x=values[f'f{quad}'][fail_satellite][0],
                         y=values[f'f{quad}'][fail_satellite][1],
                         ax=ax, color='lightgrey')
    for satellite in values[quad]:
        g = sns.lineplot(x=values[quad][satellite][0],
                         y=values[quad][satellite][1],
                         ax=ax)
    g.set_title(quadrant_labels[i])
    g.set_ylabel('volts/volts')
    g.set_xlabel('reflector height (m)')
    

In [ ]:
# plotting the qc metrics graphs
success, fail = gnssrefl_helpers.quicklook_metrics(metrics)
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(10,10), sharex=True)
fig.suptitle(f'QuickLook Retrieval Metrics: {station} GPS L1', size=16)

for i, ax in enumerate(axes):
    g = sns.scatterplot(x='Azimuth',y=success.columns[i+1], data=success, ax=ax, label='good')
    g = sns.scatterplot(x='Azimuth',y=fail.columns[i+1], data=fail, ax=ax, color='lightgrey', label='bad')
    
axes[0].legend(loc='upper right')

avg_rh = np.mean(success['Reflector Height'])
print(f'Average reflector height value: {avg_rh:.1f}')

plt.tight_layout()
plt.show()

Looking at the metrics plots, do you have some ideas on how to change the azimuth mask angles?

Use whichever quicklook plotting method to compare the above to its level when the site was installed in the year 2011 in the cell provided below (run rinex2snr and then quicklook similar to what was shown above).

In [ ]:
# place code here



Now, lets make SNR files for the whole year 2012: We will use the 'weekly' parameter that will make just one day of the week over the period we give it. This is in the interest of saving time. It should take ~5 minutes to complete.

In [ ]:
run_gnssrefl.rinex2snr(station, year, 1, doy_end=366, weekly=True)

We will next analyze a year of L1 GPS reflection data from this site. We will use the default minimum and maximum 
reflector height values (0.5 and 6 meters). But for the reasons previously stated, we will set a minimum elevation angle of 7 degrees. We also specify that we only want to use the L1 data. We use the utility make_json_input to set and store these analysis settings:

In [ ]:
# run this cell to view the available parameters and current defaults
run_gnssrefl.make_json?

In [ ]:
# We decided the best values for these using the QC metric plots above. Do these make sense based on those plots above?
min_elevation_angle = 7
peak_to_noise = 3
amplitude = 8
run_gnssrefl.make_json(station, lat, long, height, e1=min_elevation_angle, peak2noise=peak_to_noise, ampl=amplitude, l1=True)

This is the json file we created:

In [ ]:
# This is the json file that was created with the defaults/parameters you set above
json_file = f'{refl_code_loc}/input/{station}.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)
    
file

Now we are going to hand edit the azimuths to between 40 and 330 degrees. Was this close to your estimate using the QC plots?

In [ ]:
# Now lets edit the json file
with open(json_file, "r") as myfile:
    file = json.load(myfile)
    
# Here is where we can 'hand edit' values in the json file
# lets edit the azimuths. We set these values by looking at the metrics qc plot above
file['azval'] = [40,90,90,180,180,270,270,330]
os.remove(json_file)

with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)
    
# now lets view it again and note the difference
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

Now that you have SNR files and json inputs, you can go ahead and estimate reflector heights for the year 2012:

*note that it will be normal to see 'Could not read the first SNR file:' results - this is because we used the weekly setting when downloading the snr files. We are setting gnssir to run for every day of the year but if the snr file doesn't exist, it will continue on - in this case we only have one snr file per week.

In [ ]:
run_gnssrefl.gnssir?

In [ ]:
year = 2012
doy = 1
doy_end = 366
run_gnssrefl.gnssir(station, year, doy, doy_end=doy_end, screenstats=False)

Now, we can use the daily_avg tool to compute a daily average reflector height. A median filter is set to 0.25 meters 
and 30 individual tracks are required in order to recover a daily average:

In [ ]:
run_gnssrefl.daily_avg(station, medfilter=.25, ReqTracks=30, plt2screen=False, txtfile=f'{station}-dailyavg.txt')

This will create a daily file that contains the daily averages. Let's plot them:

In [ ]:
filepath = f'{refl_code_loc}/Files/{station}-dailyavg.txt'

# I have provided  a function that will read the file for you.
data = gnssrefl_helpers.read_allrh_file(filepath)
df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])
df.head()

In [ ]:
plt.figure(figsize=(8,8))
g = sns.scatterplot(x='dates', y='rh', data=df, legend=False)
# here we flip the axis so its low reflector height (higher snow) to higher reflector height (lower snow)
g.set_ylim(3.6,2.3)
g.set_ylabel('Reflector Height (m)');

The data in this plot show you long-term accumulation as well as relatively small snow accumulation events. The overall plot is dominated by the large melt event in the summer.